<a href="https://colab.research.google.com/github/ewotawa/secure_private_ai/blob/master/Section_3_Securing_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

### Install libraries and dependencies

In [108]:
# PySyft

!pip install syft

import syft as sy

# PyTorch

!pip install torch
!pip install torchvision

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import torchvision
from torchvision import datasets, transforms

# Numpy

import numpy as np

# time

import time

# Project: Federated Learning with a Trusted Aggregator

In [0]:
# try this project here!

### Instructor's Work

In [110]:
import syft as sy
import torch
hook = sy.TorchHook(torch)
from torch import nn, optim

W0724 21:32:27.047925 140090936674176 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
# create a few workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [112]:
# inform each worker that the other workers exist

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([bob, alice])

W0724 21:32:27.084788 140090936674176 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0724 21:32:27.088282 140090936674176 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0724 21:32:27.090273 140090936674176 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0724 21:32:27.093098 140090936674176 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0724 21:32:27.094432 140090936674176 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0724 21:32:27.097769 140090936674176 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# Same Toy Dataset and simple linear model

data = torch.tensor([[0,0], [0,1], [1,0], [1,1.]], requires_grad=True)
target = torch.tensor([[0], [0], [1], [1.]], requires_grad=True)

In [0]:
# get pointers to training data on each worker by sending some training data to bob and alice

bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
# initialize a Toy Model

model = nn.Linear(2, 1)

In [0]:
# instead of having one model, have two different models that we send to the two different workers so that they can be averaged.

bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [0]:
# create two separate optimizers

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

In [118]:
# train the models

for round_iter in range(10):
  
  bobs_model = model.copy().send(bob)
  alices_model = model.copy().send(alice)
  
  bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
  alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)
  
  for i in range(10):
    
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target) **2).sum()
    bobs_loss.backward()
    
    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    bobs_loss
    
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target) **2).sum()
    alices_loss.backward()
    
    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss
  
  bobs_model.move(secure_worker)
  alices_model.move(secure_worker)
  
  with torch.no_grad():
    weights = model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    bias = model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
  
  secure_worker.clear_objects()
  
  print("Iteration:\t", round_iter)
  print("Bob:\t", str(bobs_loss), "\tAlice:\t", str(alices_loss))
  print("Weights: ", str(weights), "\nBias: ", str(bias),"\n")

Iteration:	 0
Bob:	 tensor(0.0172) 	Alice:	 tensor(0.0478)
Weights:  Parameter containing:
tensor([[0.0520, 0.2233]], requires_grad=True) 
Bias:  Parameter containing:
tensor([0.1860], requires_grad=True) 

Iteration:	 1
Bob:	 tensor(0.0008) 	Alice:	 tensor(0.0139)
Weights:  Parameter containing:
tensor([[0.2214, 0.0932]], requires_grad=True) 
Bias:  Parameter containing:
tensor([0.2514], requires_grad=True) 

Iteration:	 2
Bob:	 tensor(0.0003) 	Alice:	 tensor(0.0042)
Weights:  Parameter containing:
tensor([[0.3415, 0.0315]], requires_grad=True) 
Bias:  Parameter containing:
tensor([0.2522], requires_grad=True) 

Iteration:	 3
Bob:	 tensor(0.0010) 	Alice:	 tensor(0.0015)
Weights:  Parameter containing:
tensor([[0.4360, 0.0048]], requires_grad=True) 
Bias:  Parameter containing:
tensor([0.2332], requires_grad=True) 

Iteration:	 4
Bob:	 tensor(0.0013) 	Alice:	 tensor(0.0007)
Weights:  Parameter containing:
tensor([[ 0.5140, -0.0059]], requires_grad=True) 
Bias:  Parameter containing:
te

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [120]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [121]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [122]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [123]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [124]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
# try this project here!

### My work

Create a method called <strong>encrypt()</strong>  
<ul>
  <li>Accepts two input parameters:
    <ul>
      <li>Number to be encrypted</li>
      <li>Number of shares to be split into</li>
    </ul>
  </li>
  <li>Returns a tuple of shares</li>
</ul>

Create a method called <strong>decrypt()</strong>  
* Accepts as input a tuple of shares 
* Returns the decrypted value 

Create a method called <strong>add()</strong> 
* Accepts two tuples of shares 
* Returns a single tuple of shares which are added correctly according to the additive secret sharing protocol.

In [0]:
def encrypt(orig_number, num_shares, Q):
  x = int(orig_number)
  
  # Q = 23740629843760239486723 #large prime number
  
  shares = []
  share_sum = int()
  
  for i in range(num_shares - 1):
    share = torch.randn(1) * 100000000000
    shares.append(int(share.item()))
  
  last_share = Q - int(sum(shares)) + x
  shares.append(last_share)
  
  sum_out = sum(shares)
  
  shares = tuple(shares)
  
  # validation:
  # print(sum_out % Q)
  
  return(shares)

In [0]:
def decrypt(encrypted_shares, Q):
  sum_shares = sum(encrypted_shares)
  decrypted = sum_shares % Q
  return decrypted

In [129]:
Q = 23740629843760239486723 #large prime number 
shares = encrypt(5, 200, Q)
print(shares)
decrypted = decrypt(shares, Q)
print(decrypted)

(-16299107328, 16417369088, -87592714240, -16138979328, 97785233408, -75280007168, -11164367872, 9505028096, -5318636032, 119584915456, -29610129408, -84420255744, -53044461568, -66198859776, -176867803136, -45476708352, -35281326080, 127880822784, 23148783616, 77031047168, -288498221056, -51866673152, -104891056128, 100198334464, 48427098112, -122550591488, -18821804032, -94069440512, 165857132544, -34860232704, -53949571072, 194290614272, 188169125888, 214135734272, 47238971392, -61646286848, 5192459776, 17673449472, 49621430272, -65583263744, 199062110208, 265959276544, 66966253568, 82215534592, -138244014080, 20322711552, -11248312320, -57689669632, -45540872192, -24303663104, 148596850688, -169381871616, 67125932032, -67575627776, -155408334848, -39351701504, -49157603328, 114195079168, 127995027456, 23084216320, -20253329408, 127976472576, 119955021824, -80149897216, 119349608448, 71245840384, -8270036480, -103439720448, 30777206784, 74971840512, -53201084416, 132974714880, -5934

In [0]:
def add(tuple1, tuple2):
  sum_tuple = tuple1 + tuple2
  return sum_tuple

In [131]:
tuple1 = encrypt(5, 5, Q)
tuple2 = encrypt(10, 5, Q)

sum_tuple = add(tuple1, tuple2)
print(sum_tuple)

decrypt(sum_tuple, Q)

(123663687680, -12077692928, 19199176704, 11176441856, 23740629843618277873416, -85316141056, -102852673536, -76582682624, 2124237568, 23740629844022866746381)


15

### Instructor's work

In [0]:
import random

In [0]:
Q = 23740629843760239486723 #large prime number 

In [0]:
def encrypt(x, n_shares):
  shares = list()
  for i in range(n_shares - 1):
    shares.append(random.randint(0, Q))
  final_share = Q - (sum(shares) % Q) + x
  shares.append(final_share)
  return tuple(shares)

In [135]:
encrypt(5, n_shares=10)

(2854763494207068320741,
 3529569098382468296771,
 2294435248301830186554,
 2434768716663077378205,
 6853080886763810499533,
 18336730970245194223018,
 4053270711496065939998,
 17180375321928894169655,
 19636594857393027052230,
 17788930069659521880192)

In [0]:
def decrypt(shares):
  return sum(shares) % Q

In [137]:
decrypt(encrypt(5, 10))

5

In [0]:
def add(a, b):
  c = list()
  assert(len(a) == len(b))
  for i in range(len(a)):
    c.append((a[i] + b[i]) % Q)
  return tuple(c)

In [139]:
decrypt(add(encrypt(5, 10), encrypt(5, 10)))

10

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [142]:
encode(3.5)

35000

In [143]:
decode(35000)

3.5

In [146]:
x = encrypt(encode(5.5), 3)
y = encrypt(encode(2.3), 3)
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = torch.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice, secure_worker)

In [151]:
bob._objects

{85389643982: tensor([ 765499672199538488, 2252359132134492253, 2366545669795321936,
         2638457181889128042, 2770828211024011279])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [153]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:40287335887 -> bob:93993341461]
	-> (Wrapper)>[PointerTensor | me:89385662884 -> alice:68538084216]
	-> (Wrapper)>[PointerTensor | me:84343111995 -> secure_worker:38758458912]
	*crypto provider: me*

In [154]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = torch.tensor([0.1,0.2,0.3])

In [157]:
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [159]:
x.child.child

tensor([100, 200, 300])

In [0]:
y = x + x

In [161]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = torch.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [0]:
y = x + x

In [166]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Build on the first project, where you perform federated learning with a trusted, secure aggregator.

<ul>
  <li>Take the same neural network you used in the first project. </li>
  <li>Aggregate gradients using additive secret sharing and fixed-precision encoding.  </li>
  <li>Use at least three data owners per aggregation.  
    <ul>
      <li>Ensures that no one will ever see anyone's gradients other than their own. Protects privacy without needing to trust a secure aggregator.  </li>
    </ul>
  </li>
</ul>  